# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# Import API key
from api_keys import g_key

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
city_weather_df = pd.read_csv("../output_data/cities.csv")
city_weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Podosinovets,60.2776,47.0659,8.87,94,99,2.98,RU,1670095756
1,1,Castro,-24.7911,-50.0119,71.94,85,100,1.77,BR,1670095757
2,2,East London,-33.0153,27.9116,66.34,75,100,5.86,ZA,1670095757
3,3,Hermanus,-34.4187,19.2345,60.80,63,99,8.55,ZA,1670095757
4,4,Port Elizabeth,-33.9180,25.5701,65.10,85,20,2.30,ZA,1670095758


In [8]:
city_weather_df.count()

Unnamed: 0    93
City          93
Lat           93
Lng           93
Max Temp      93
Humidity      93
Cloudiness    93
Wind Speed    93
Country       92
Date          93
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key = g_key)
lat_lng = city_weather_df[['Lat','Lng']]
humidity = city_weather_df['Humidity'].astype(float)

In [12]:
#create the heatmap
heatmap = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_lng,weights=humidity,dissipating=False, max_intensity=100,                                point_radius = 1)
heatmap.add_layer(heat_layer)
heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#weather conditions:
#1.max temp < 80 but > 70
#2.wind speed < 10 mph
#3.zero cloudiness
#drop any rows that do not satisfy all three conditions

perf_temp = city_weather_df.loc[(city_weather_df['Max Temp'] < 80)& 
                                (city_weather_df['Max Temp'] > 70)&
                                (city_weather_df['Wind Speed'] < 10)&
                                (city_weather_df['Cloudiness'] == 0)]

perf_temp.count()

Unnamed: 0    0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [14]:
#drop rows with null values
perf_temp = perf_temp.dropna(how='any')
perf_temp.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
#add a hotel name column 
perf_temp['Hotel Name'] = ''
perf_temp.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [19]:
#smaller sample size to avoid overuse of the Google Maps API call limits
# perf_temp_reduced = perf_temp.iloc[0:10,:]
# perf_temp_reduced

##for each city, use Google Places API to find first hotel located within
#5,000 meters of the coordinates

#look at airport ratings lesson for instruction, and use iterrows
# set up a parameters dictionary
params = {
    "radius": 5000,
    "types": 'lodging',
    "keyword": "Hotel",
    "key": g_key
}
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in perf_temp.iterrows():
    lat= row['Lat']
    lng= row['Lng']
    params['location'] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params).json()
    try:
        perf_temp.loc[index,'Hotel Name'] = hotel_name['results'][0]['name']
    except:
        perf_temp.loc[index,'Hotel Name'] = 'No Hotel'



In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:

#create the heatmap
hotel_fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(lat_lng,weights=humidity,dissipating=False, max_intensity=100,
#                                  point_radius = 1)
hotel_fig.add_layer(heat_layer)
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
hotel_fig.add_layer(marker_layer)
# Display figure
hotel_fig